In [37]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [38]:
!pip install nltk scikit-learn pandas matplotlib


In [39]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [111]:

import pandas as pd

url_test = "https://raw.githubusercontent.com/KJanzon/lab-natural-language-processing/refs/heads/main/data/kg_test.csv"
url_train = "https://raw.githubusercontent.com/KJanzon/lab-natural-language-processing/refs/heads/main/data/kg_train.csv"


In [128]:
from sklearn.model_selection import train_test_split


## Read Data for the Fraudulent Email Kaggle Challenge
train_data = pd.read_csv(url_train)
test_data = pd.read_csv(url_test)

# Reduce the training set to speed up development.
# Modify for final system
train_data = train_data.head(1000)
print(train_data.shape)
train_data.fillna("",inplace=True)
print(train_data.head)


(1000, 2)
<bound method NDFrame.head of                                                   text  label
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                             Will do.      0
2    Nora--Cheryl has emailed dozens of memos about...      0
3    Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                  fyi      0
..                                                 ...    ...
995  So what's the latest? It sounds contradictory ...      0
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...      1
997  Barb I will call to explain. Are you back in t...      0
998    Yang on travelNot free tonite.May work tomorrow      0
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...      0

[1000 rows x 2 columns]>


<ipython-input-128-c6cb4fc71a28>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.fillna("",inplace=True)


In [129]:
# Your code# Ensuring the dataset has the expected structure
if "text" in train_data.columns and "label" in train_data.columns:
    X = train_data["text"]  # Features (SMS messages)
    y = train_data["label"]  # Labels (Spam or Ham)
else:
    X = train_data.iloc[:, 0]  # Assuming first column is text
    y = train_data.iloc[:, 1]  # Assuming second column is label


In [85]:
# Download NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Data Preprocessing

In [130]:
import re

def clean_html(text):
    # Remove inline JavaScript & CSS
    text = re.sub(r'<script.*?</script>', '', text, flags=re.DOTALL)  # Remove JavaScript
    text = re.sub(r'<style.*?</style>', '', text, flags=re.DOTALL)  # Remove CSS

    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)

    return text

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [131]:
def clean_text(text):
    import re
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = re.sub(r'\b\d+\b', '', text)  # Remove standalone numbers
    text = re.sub(r'\d+', '', text)  # Remove numbers inside words
    text = re.sub(r'\b[a-zA-Z]\b', '', text)  # Remove single characters
    text = re.sub(r'\b\w{20,}\b', '', text)  # Remove long words (20+ chars)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text.lower()


## Now let's work on removing stopwords
Remove the stopwords.

In [69]:
# Load English stopwords
stop_words = set(stopwords.words('english'))

# Manually add unwanted words
custom_stopwords = {"u", "pm", "mr", "percent"}
stop_words.update(custom_stopwords)  # Add to stopword set

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [70]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Apply lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a sentence
    return ' '.join(lemmatized_words)

### Let's divide the training and test set into two partitions

In [132]:


# Apply text preprocessing BEFORE splitting
X_cleaned = X.apply(clean_html)          # Remove HTML
X_cleaned = X_cleaned.apply(clean_text)   # Remove special characters, numbers, etc.
X_cleaned = X_cleaned.apply(remove_stopwords) # Remove stopwords
X_cleaned = X_cleaned.apply(lemmatize_text)   # Apply Lemmatization


    # Split dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2, random_state=42)

# Display dataset partition sizes
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


Training set size: 800
Testing set size: 200


In [133]:
print("before", X.head())

print("after", X_train.head())

before 0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
Name: text, dtype: object
after 29         regard nelson smithkindly reply private email
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


In [50]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer

snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [134]:
from collections import Counter


# Separate ham and spam messages
ham_messages = X_train[y_train == 0]  # Ham (Non-spam)
spam_messages = X_train[y_train == 1]  # Spam

# Function to get top N words, excluding stopwords
def get_top_words(messages, n=10):
    all_words = ' '.join(messages).split()  # Merge all messages and split into words
    filtered_words = [word for word in all_words if word.lower() not in stop_words]  # Remove stopwords
    word_counts = Counter(filtered_words)  # Count word frequencies
    return word_counts.most_common(n)  # Get top n words

# Get top 10 words in ham and spam messages
top_ham_words = get_top_words(ham_messages, 10)
top_spam_words = get_top_words(spam_messages, 10)

# Convert to DataFrame for better visualization
df_top_words = pd.DataFrame({
    "Top Ham Words": [word[0] for word in top_ham_words],
    "Ham Count": [word[1] for word in top_ham_words],
    "Top Spam Words": [word[0] for word in top_spam_words],
    "Spam Count": [word[1] for word in top_spam_words]
})

# Display results
print(df_top_words.head(10))  # Display the top 10 words in both categories

  Top Ham Words  Ham Count Top Spam Words  Spam Count
0         would         93          money         795
1         state         92        account         662
2     president         84           bank         606
3          call         73           fund         565
4     secretary         71       business         393
5          time         70    transaction         347
6          work         63        country         337
7           one         63       transfer         326
8          said         59        company         318
9      american         58        million         315


In [74]:
# Find all ham (non-spam) messages that contain the word "president"
ham_with_president = X_cleaned[(y_train == 0) & (X_cleaned.str.contains(r'\bpresident\b', case=False, na=False))]

# Display the first few results
print("Ham Messages Containing 'President':")
print(ham_with_president.head(10))  # Show first 10 messages


Ham Messages Containing 'President':
64     sbwhoeopwednesday september amhh fyi midterm s...
112    madame secretary forgive email wanted thank pr...
189    interest hague latest briefing hostile tony to...
191    good morning cheryl trying reach followup week...
242    sbwhoeopmonday november pmhfyi chance wm hague...
269    spanish fm moratinos requested speak today tom...
304    attached enclosed demarche letter went reflect...
326    httpwwwnybookscomarticlesarchivesdecbitterneww...
372    depart private residence en route white house ...
391    apparently cant get away sharing ft oped page ...
Name: text, dtype: object


In [90]:
# Ensure text preprocessing is applied
train_data["preprocessed_text"] = train_data["text"].apply(clean_text)
test_data["preprocessed_text"] = test_data["text"].apply(clean_text)

## Extra features

In [78]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_data['money_mark'] = train_data['preprocessed_text'].str.contains(money_simbol_list)*1
train_data['suspicious_words'] = train_data['preprocessed_text'].str.contains(suspicious_words)*1
train_data['text_len'] = train_data['preprocessed_text'].apply(lambda x: len(x))

test_data['money_mark'] = test_data['preprocessed_text'].str.contains(money_simbol_list)*1
test_data['suspicious_words'] = test_data['preprocessed_text'].str.contains(suspicious_words)*1
test_data['text_len'] = test_data['preprocessed_text'].apply(lambda x: len(x))

test_data.head()

,text,preprocessed_text,money_mark,suspicious_words,text_len
0,usiness is for the fact that the deceased man ...,usiness is for the fact that the deceased man ...,1,1,1294
1,They are happy to adjust to the afternoon. I a...,they are happy to adjust to the afternoon am g...,1,0,153
2,Lael Brainard was confirmed 78-19 this afterno...,lael brainard was confirmed this afternoonmigu...,1,0,174
3,H <hrod17@clintonemail.com>Friday March 26 201...,hrodclintonemailcomfriday march amsbwhoeop rei...,1,0,79
4,"n;""> Dear Good Friend,<br><br><br>I am happy t...",dear good friendbrbrbri am happy to inform you...,1,1,1320


## How would work the Bag of Words with Count Vectorizer concept?

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training set
X_train_bow = vectorizer.fit_transform(train_data["preprocessed_text"])

# Transform the test set (using the same vocabulary)
X_test_bow = vectorizer.transform(test_data["preprocessed_text"])

# Convert BoW to a DataFrame for visualization
df_train_bow = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Show first few rows of transformed dataset
print("Bag of Words Feature Matrix (Training Data):")
print(df_train_bow.head())


Bag of Words Feature Matrix (Training Data):
   aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0    0           0    0    0              0       0             0        0   
1    0           0    0    0              0       0             0        0   
2    0           0    0    0              0       0             0        0   
3    0           0    0    0              0       0             0        0   
4    0           0    0    0              0       0             0        0   

   abachace  abachaco  ...  zongo  zongothe  zuhair  \
0         0         0  ...      0         0       0   
1         0         0  ...      0         0       0   
2         0         0  ...      0         0       0   
3         0         0  ...      0         0       0   
4         0         0  ...      0         0       0   

   zulatoebikozulatonetscapeenet  zulatoffffffffffffffffff  zuma  zumac  \
0                              0                         0     0      0   
1    

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', min_df=10, max_features=3000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Train Feature Count: {X_train_tfidf.shape[1]}")
print(f"Test Feature Count: {X_test_tfidf.shape[1]}")


Train Feature Count: 1018
Test Feature Count: 1018


In [144]:
# Tokenize training and test sets
train_words = set(" ".join(X_train).split())
test_words = set(" ".join(X_test).split())

# Find words that exist in test but not in train
new_test_words = test_words - train_words

print(f"Unique words in X_test but not in X_train: {len(new_test_words)}")
print(list(new_test_words)[:20])  # Show sample words


Unique words in X_test but not in X_train: 2288
['toassist', 'aiash', 'foremostc', 'ifonly', 'chequeget', 'laszczychlstategov', 'diversiondue', 'placedthis', 'weband', 'anexcess', 'blinder', 'theprivate', 'attentioncfirstlyc', 'tocharity', 'shy', 'circlesc', 'itduring', 'wittes', 'notentertain', 'mecwe']


In [142]:
print("Sample X_train Text:")
print(X_train.head())

print("\nSample X_test Text:")
print(X_test.head())


Sample X_train Text:
29         regard nelson smithkindly reply private email
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object

Sample X_test Text:
521    dear sirc wish go offer consider partnerei mre...
737    take mind balkan second see great plug global ...
740                               pls keep update coming
660    christ bethel hospital rue aboboteabidjanivory...
411    sbwhoeopfriday february amhre bravo brava issu...
Name: text, dtype: object


## And the Train a Classifier?

In [143]:
print(f"X_train Shape: {X_train.shape}")
print(f"y_train Shape: {y_train.shape}")

X_train Shape: (800,)
y_train Shape: (800,)


In [145]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a Naïve Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred = nb_model.predict(X_test_tfidf)

# Evaluate model performance
print("🔹 Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.96       125
           1       0.91      0.95      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.95      0.94       200
weighted avg       0.95      0.94      0.95       200

Accuracy: 0.9450


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).